In [11]:
# Welcome to your new notebook
# Type here in the cell editor to add code!

df = spark.sql("SELECT * FROM lh_youtube.silver.news_data")
display(df)

SynapseWidget(Synapse.DataFrame, a73b7fe0-a3e8-4aa9-a14b-cb2ace8e6fc9)

In [12]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, 98c972f2-9778-46b6-8de6-cf89699343fa, 14, Finished, Available, Finished)

In [13]:
model= (AnalyzeText()
        .setTextCol("description")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error"))
result=model.transform(df)

StatementMeta(, 98c972f2-9778-46b6-8de6-cf89699343fa, 15, Finished, Available, Finished)

In [14]:
# result.show()
from pyspark.sql.functions import col
sentiment_df=result.withColumn("sentiment",col("response.documents.sentiment"))
sentiment_final_df=sentiment_df.drop("response","error")
sentiment_final_df.show()

StatementMeta(, 98c972f2-9778-46b6-8de6-cf89699343fa, 16, Finished, Available, Finished)

+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------+---------+
|               title|         description|            category|                 url|               image|             source|datepublished|sentiment|
+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------+---------+
|India vs South Af...|India vs South Af...|            [sports]|https://news.abpl...|https://feeds.abp...|           Abp News|  22-Feb-2026|    mixed|
|SL vs ENG: Rashid...|Rain threatens to...|            [sports]|https://timesofin...|https://static.to...| The Times Of India|  22-Feb-2026| negative|
|Revolutionizing D...|Revolutionizing D...|   [technology, top]|https://www.devdi...|https://devdiscou...|       Devdiscourse|  21-Feb-2026|  neutral|
|Tesla begins hiri...|According to the ...|   [technology, top]|https://www.busin...|https://a

In [15]:
from pyspark.sql.functions import to_date,col
sentiment_final_df=sentiment_final_df.withColumn("datepublished",to_date(col("datepublished"),"dd-MMM-yyyy"))

StatementMeta(, 98c972f2-9778-46b6-8de6-cf89699343fa, 17, Finished, Available, Finished)

In [16]:
from pyspark.sql.utils import AnalysisException
try:
    table_name= "silver.news_with_sentiment_data"
    sentiment_final_df.write.format("delta").saveAsTable(table_name)
except AnalysisException:
    print("Table already exists")

    sentiment_final_df.createOrReplaceTempView("vw_sentiment_final_df")
    
    spark.sql(f""" MERGE INTO {table_name} target_table
                    USING vw_sentiment_final_df view_table
                    ON target_table.url=view_table.url
                    WHEN MATCHED AND
                    target_table.title<>view_table.title OR
                    target_table.description<>view_table.description OR
                    target_table.category<>view_table.category OR
                    target_table.image<>view_table.image OR
                    target_table.source<>view_table.source OR
                    target_table.datepublished<>view_table.datepublished
                    THEN UPDATE SET *
                    WHEN NOT MATCHED THEN INSERT * """)

StatementMeta(, 98c972f2-9778-46b6-8de6-cf89699343fa, 18, Finished, Available, Finished)

Table already exists
